In [1]:
# Determine start time
import time
#start = time.time()

import datetime
import json

In [2]:
def getData():
    f = open('binpack.json','r')
    x = json.load(f)
    f.close()
    for i in range(len(x)):
        myData = x[i]['data']
        x[i]['data'] = {}
        for j in range(len(myData)):
            x[i]['data'][j] = myData[j] 
    return x



{"cap":cap,"data":{}}

In [3]:
def checkCapacity(articles, bin_contents, bin_cap):
    """ articles: a dictionary of the items to be loaded into the bins: the key is the article id and the value is the article volume """
    """ bin_contents is expected to be a list of lists, where each sub-list is the contents of each bin vis article ids  """
    """ bin_cap: capacity of each of the identical bins """
    """ This function returns two parameters, the first of which is the number of bins that are within capacity and, the second, the number of overloaded bins """
    
    num_ok = 0
    num_over = 0
    if isinstance(articles,dict):
        if isinstance(bin_contents,list):
            item_key_good = True
            for this_bin in bin_contents:
                if isinstance(this_bin,list):
                    load = 0.0
                    for this_item in this_bin:
                        if this_item not in items.keys():
                            item_key_good = False
                        else:
                            load += articles[this_item]
                    if item_key_good == False:
                        print("function checkCapacity(), bad item key")
                        return 'bad_key', 'bad_key'
                    elif load <= bin_cap:
                        num_ok += 1
                    else:
                        num_over += 1
                else:
                    print("function checkCapacity(),contents of each bin must be in a sub-list")
                    return 'sublist_error','sublist_error'
            return num_ok, num_over
        else:
            print("function checkCapacity(), bin_contents must be in a list")
            return 'list_needed', 'list_needed'
    else:
        print("function checkCapacity(), articles argument requires a dictionary")
        return 'dict_needed', 'dict_needed'
 

In [4]:
def checkAllPoints(articles, bin_contents):
    """ Check to be sure that all items are packed in one bin """
    
    err_mess = ""
    err_mult= False
    checkit = {}
    for this_bin in bin_contents:
        for this_art in this_bin:
            checkit[this_art] = checkit.get(this_art,0) + 1
            if checkit[this_art] > 1:
                err_mult = True
                err_mess += "Loc assigned mult times"
                
    err_all = False
    for key_art in articles.keys():
        if key_art not in checkit.keys():
            err_all = True
            err_mess += "Some locs not assigned to bins"
            
    return err_mult, err_all, err_mess

In [25]:
import math
#bin_cap = 1
#articles = {0:0.5,1:0.7,2:0.5,3:0.2,4:0.4,5:0.2,6:0.5,7:0.1,8:0.6}
print("data 4",getData()[4])
#bin_cap = 100.0
#data= [54.9763980770002, 45.0029239348998, 61.3426025372739, 80.0, 77.6012305922644, 78.2923783726743, 
#                        23.3518777532423, 39.8208126512123, 68.94959337016, 10.4524154059541, 60.4772617438057, 10.0, 
#                        79.4241251161082, 50.5316063003593, 10.0, 44.552677715225, 72.8685047807319, 10.0, 10.0, 
#                        46.3487596059364, 10.0, 73.0560543111617]
bin_cap=getData()[4]["cap"]
data = getData()[4]["data"]
articles = {}
print(len(data))
for i in range(len(data)):
    articles[i]=data[i]
items_list = [(k,v) for k,v in articles.items()]
items_list.sort(key=lambda x:x[1],reverse=True)
print(items_list)
lb = math.ceil(sum([v for k,v in items_list])/bin_cap)
print("lb:",lb)
bin_contents = [[] for i in range(lb)]
caplist1 = [bin_cap for i in range(lb)]
for k,v in items_list:
    #k = items_list[i][0]
    #v = items_list[i][1]
    for j in range(len(caplist1)):
        l = [float(caplist1[i] - v) for i in range(len(caplist1))]
        if len([i for i in l if i>=0]) > 0:
            min_indx = l.index(min([i for i in l if i>=0])) # find the index of the minimal positive number
            #print("min index",min_indx)
            #print("l list:",l)
            caplist1[min_indx] = caplist1[min_indx]-v
            bin_contents[min_indx].append(k)
            #print(bin_contents)
        else:
            bin_contents.append([k])
            caplist1.append(bin_cap-v)
        
        break
print(bin_contents)
print(caplist1)

data 4 {'cap': 100.0, 'data': {0: 99.975, 1: 0.025, 2: 99.9, 3: 0.1, 4: 50.0, 5: 50.0, 6: 50.0, 7: 50.0, 8: 50.0, 9: 50.0, 10: 50.0, 11: 50.0, 12: 50.0, 13: 50.0, 14: 50.0, 15: 50.0, 16: 85.0, 17: 15.0, 18: 75.0, 19: 25.0, 20: 90.0, 21: 10.0}}
22
[(0, 99.975), (2, 99.9), (20, 90.0), (16, 85.0), (18, 75.0), (4, 50.0), (5, 50.0), (6, 50.0), (7, 50.0), (8, 50.0), (9, 50.0), (10, 50.0), (11, 50.0), (12, 50.0), (13, 50.0), (14, 50.0), (15, 50.0), (19, 25.0), (17, 15.0), (21, 10.0), (3, 0.1), (1, 0.025)]
lb: 11
[[0, 1], [2], [20, 21], [16, 17], [18, 19], [4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14, 15], [3]]
[5.68295410730002e-15, 0.09999999999999432, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 99.9]


In [90]:
def binpack(articles,bin_cap):
    """ Write your heuristic bin packing algorithm in this function using the argument values that are passed
             articles: a dictionary of the items to be loaded into the bins: the key is the article id and the value is the article volume
             bin_cap: the capacity of each (identical) bin (volume)
    
        Your algorithm must return two values as indicated in the return statement:
             bin_contents: A list of lists, where each sub-list indicates the dictionary keys for the items assigned to each bin.      
                           Each sublist contains the item id numbers (integers) for the items contained in that bin.  The integers refer to 
                           keys in the items dictionary. 
   """
    bin_contents = []    # use this list document the article ids for the contents of 
                         # each bin, the contents of each is to be listed in a sub-list
   
    #max_bin = -1
    bins = {}      # dictionary to store items in each bin, each dictionary entry is a bin, its value is a tuple with the current volume and a list of the item ids
    
    items_list = [(k,v) for k,v in articles.items()]
    items_list.sort(key=lambda x:x[1],reverse=True)
    
    """ Insert code here.  You may adjust lines 93-94 (above) or 99-100 (below) if needed
    """
    import math
    lb = math.ceil(sum([v for r, v in items_list])/bin_cap)
    #need a set of variables to keep track of each bin's capacity
    binlist = [] # keep track of bins
    caplist = [bin_cap for i in range(lb)]
    for i in range(len(items_list)):
        k = items_list[i][0]
        v = items_list[i][1]
        for j in range(len(caplist)):
            if v <= caplist[j]:
                caplist[j] = caplist[j]-v
                if j >= len(bin_contents):
                    bin_contents.append([k])
                else:
                    bin_contents[j].append(k)
                break
    #############################
    #for this_key in bins.keys():
    #    bin_contents.append(bins[this_key][1])
    
    # Print the contents of the different bins.
    # If you comment out the print line below, then you will only have total 
    # number of bins.
    return bin_contents

In [91]:
binpack(articles,bin_cap)

[(1, 0.7), (8, 0.6), (0, 0.5), (2, 0.5), (6, 0.5), (4, 0.4), (3, 0.2), (5, 0.2), (7, 0.1)]


[[1, 3, 7], [8, 4], [0, 2], [6, 5]]

In [19]:
""" Get solutions based on submission """
probData = getData()
problems = range(len(probData)) 
silent_mode = False    # use this variable to turn on/off appropriate messaging depending on student or instructor use

for problem_id in problems:
    bin_cap = probData[problem_id]['cap']
    items = probData[problem_id]['data']
    #finished = False
    errors = False
    response = None
    
    #while finished == False:
    response = binpack(items,bin_cap)
    #if not isinstance(response,str):
    if isinstance(response,list):
        num_ok, num_over = checkCapacity(items, response, bin_cap)
        if not isinstance(num_ok,int) or not isinstance(num_over,int):
            errors = True
            if silent_mode:
                status = num_ok
            else:
                print("P"+str(problem_id)+num_ok+"_")
                
        err_mult, err_all, err_mess = checkAllPoints(items, response)
        if err_mult or err_all:
            errors = True
            if silent_mode:
                status += "_" + err_mess
            else:
                print("P"+str(problem_id)+err_mess+"_")
    else:
        errors = True
        if silent_mode:
            status = "response not a list"
        else:
            print("P"+str(problem_id)+"reponse_must_be_list_")
            
    if errors == False:
        
        if silent_mode:
            status = "P"+str(problem_id)+"bin_pack_"
        else:
            print("Bins Packed for Problem ", str(problem_id)," ....")
        
        if silent_mode:
            print(status+"; num_ok: "+num_ok+"; num_over: "+num_over)
        else:
            print("num_ok/num_over: ", num_ok,"/",num_over)
        this_time = datetime.datetime.now()     # not use; formerly planned as input to DB
        
# Determine ending time
#end = time.time()

# Print total time.
print("For a total time in seconds of ")
#print(end - start)

[(828, 0.99998981969998), (148, 0.999242992773833), (586, 0.998872729809046), (817, 0.99844461362121), (793, 0.997385936666068), (476, 0.996912889558587), (981, 0.994862952888474), (138, 0.992858299842357), (87, 0.992799476731147), (567, 0.990787109310402), (570, 0.990455063896835), (627, 0.989154463247956), (569, 0.988188611514404), (61, 0.987402037403383), (940, 0.985116434833485), (701, 0.984029392954913), (874, 0.983843849714223), (469, 0.982929813826273), (593, 0.982295703638171), (935, 0.982170881413821), (435, 0.981254663571667), (66, 0.980713159185675), (326, 0.978283236079716), (910, 0.978279156625955), (964, 0.978052139115485), (106, 0.977687717347713), (27, 0.977190462953884), (433, 0.976527141196799), (344, 0.975334317042495), (946, 0.975148309617305), (478, 0.974093057844572), (390, 0.973361516736555), (276, 0.973108140332297), (746, 0.971816987340777), (767, 0.970936709323153), (425, 0.97016977841433), (223, 0.967967437399389), (667, 0.967207827028314), (54, 0.96663542990

In [20]:
list = [8 for i in range(4)]
print(list)

[8, 8, 8, 8]


In [8]:
dic = {"cap": 100.0, "data": [54.9763980770002, 45.0029239348998, 61.3426025372739, 80.0, 77.6012305922644, 78.2923783726743, 
                        23.3518777532423, 39.8208126512123, 68.94959337016, 10.4524154059541, 60.4772617438057, 10.0, 
                        79.4241251161082, 50.5316063003593, 10.0, 44.552677715225, 72.8685047807319, 10.0, 10.0, 
                        46.3487596059364, 10.0, 73.0560543111617]}

In [9]:
print(len(dic["data"]))
data = dic["data"]
indx_l = [4,16,8,2,10]
for indx in indx_l:
    print(indx,data[indx])
data.sort(reverse=True)

22
4 77.6012305922644
16 72.8685047807319
8 68.94959337016
2 61.3426025372739
10 60.4772617438057


In [126]:
l = [data[i]-15 for i in range(len(data))]
print(l.index(min(l)))
print(l)

11
[39.9763980770002, 30.002923934899798, 46.3426025372739, 65.0, 62.6012305922644, 63.29237837267431, 8.351877753242299, 24.8208126512123, 53.94959337016, -4.547584594045899, 45.4772617438057, -5.0, 64.4241251161082, 35.5316063003593, -5.0, 29.5526777152254, 57.8685047807319, -5.0, -5.0, 31.3487596059364, -5.0, 58.0560543111617]


In [129]:
print(min([i for i in l if i > 0]))
l.index(min([i for i in l if i > 0]))

8.351877753242299


6

In [30]:
data = {0: 99.975, 1: 0.025, 2: 99.9, 3: 0.1, 4: 50.0, 5: 50.0, 6: 50.0, 7: 50.0, 8: 50.0, 9: 50.0, 10: 50.0, 11: 50.0, 12: 50.0, 13: 50.0, 14: 50.0, 15: 50.0, 16: 85.0, 17: 15.0, 18: 75.0, 19: 25.0, 20: 90.0, 21: 10.0}
inx = [[1, 3, 21, 17, 19], [4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14, 15], [18], [16], [20], [2], [0]]
for l in inx:
    sum_value = [sum(data[i] for i in l)]
    print(100-sum_value)

TypeError: unsupported operand type(s) for -: 'int' and 'list'